In [ ]:
!pip install mlflow
!pip install findspark
!pip install pyspark

In [10]:
import findspark
import pyspark
findspark.init()

from pyspark.sql import SparkSession


spark = SparkSession.builder.getOrCreate()

24/07/30 09:26:27 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [11]:
df = spark.read.option("header","true").option("inferSchema","true").csv("./models/data/news_category_test.csv").toDF("label","text")
df.show()

+--------+--------------------+
|   label|                text|
+--------+--------------------+
|Business|Unions representi...|
|Sci/Tech| TORONTO, Canada ...|
|Sci/Tech| A company founde...|
|Sci/Tech| It's barely dawn...|
|Sci/Tech| Southern Califor...|
|Sci/Tech|"The British Depa...|
|Sci/Tech|"confessed author...|
|Sci/Tech|\\FOAF/LOAF  and ...|
|Sci/Tech|"Wiltshire Police...|
|Sci/Tech|In its first two ...|
|Sci/Tech| A group of techn...|
|Sci/Tech| Apple Computer I...|
|Sci/Tech| Free Record Shop...|
|Sci/Tech|A giant 100km col...|
|Sci/Tech|"Dolphin groups, ...|
|Sci/Tech|Tyrannosaurus rex...|
|Sci/Tech|  Scientists have...|
|Sci/Tech|  ESAs Mars Expre...|
|Sci/Tech|When did life beg...|
|Sci/Tech|update Earnings p...|
+--------+--------------------+
only showing top 20 rows



24/07/30 09:26:30 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.


In [12]:
import mlflow

loaded_model_udf = mlflow.pyfunc.spark_udf(spark, 
                                           model_uri="runs:/478eacd5b1f04010be3b633b7e4434f4/classification", 
                                           result_type='string')

2024/07/30 09:26:31 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - accelerate (current: uninstalled, required: accelerate==0.32.1)
 - torch (current: uninstalled, required: torch==2.3.1)
 - torchvision (current: uninstalled, required: torchvision==0.18.1)
 - transformers (current: uninstalled, required: transformers==4.42.4)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
2024/07/30 09:26:31 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.
2024/07/

In [13]:
test = df.select(df.text, df.label, loaded_model_udf(df.text).alias("prediction"))
test.show()

24/07/30 09:26:32 ERROR ArrowPythonRunner: Python worker exited unexpectedly (crashed)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/HP/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1225, in main
    eval_type = read_int(infile)
  File "/Users/HP/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 596, in read_int
    raise EOFError
EOFError

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.PythonArrowOutput$$anon$1.read(PythonArrowOutput.scala:118)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.ha

Py4JJavaError: An error occurred while calling o146.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 11) (macbook-air.lan executor driver): java.lang.UnsupportedOperationException: sun.misc.Unsafe or java.nio.DirectByteBuffer.<init>(long, int) not available
	at org.apache.arrow.memory.util.MemoryUtil.directBuffer(MemoryUtil.java:174)
	at org.apache.arrow.memory.ArrowBuf.getDirectBuffer(ArrowBuf.java:229)
	at org.apache.arrow.memory.ArrowBuf.nioBuffer(ArrowBuf.java:224)
	at org.apache.arrow.vector.ipc.WriteChannel.write(WriteChannel.java:133)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.writeBatchBuffers(MessageSerializer.java:303)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.serialize(MessageSerializer.java:276)
	at org.apache.arrow.vector.ipc.ArrowWriter.writeRecordBatch(ArrowWriter.java:147)
	at org.apache.arrow.vector.ipc.ArrowWriter.wr